In [ ]:
import keras
print(keras.__version__)

import tensorflow as tf
print(tf.__version__)

# from PIL import Image
# img = Image.open("img.png").getdata()
# print(len(img))

from scipy import misc
img = misc.imread("img.png")
print(img.shape)
# misc.pilutil.imshow(img). #Some issues. Fix Later


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
import numpy as np
import os

classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))

#keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

train_datagen = ImageDataGenerator(rescale = 1./255,
    rotation_range=10,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = False)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('train',
    target_size = (64, 64),
    batch_size = 2,
    color_mode = 'grayscale',
    class_mode = 'categorical')
print(training_set.class_indices)
indices = training_set.class_indices

test_set = test_datagen.flow_from_directory('test',
    target_size = (64, 64),
    batch_size = 2,
    color_mode = 'grayscale',
    class_mode = 'categorical')
print(test_set.class_indices)
indices = test_set.class_indices
f=open("targetfile","w")
f.write(str(indices))

classifier.fit_generator(training_set,
    steps_per_epoch = 1,
    epochs = 1,
    validation_data = test_set,
    validation_steps = 1)

model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
classifier.save_weights("model.h5")
print("Saved model to disk")


In [ ]:
#Neural Network visualizer

from ann_visualizer.visualize import ann_viz
#Build your model here
ann_viz(classifier)